# AMAG Car Scraper

This notebook scrapes car subscription data from AMAG (auto.amag.ch).

## Features:
- Web scraping with requests and BeautifulSoup
- Data filtering and analysis
- CSV export functionality
- Price and fuel type analysis

In [ ]:
# Install required packages
!pip install requests beautifulsoup4 pandas plotly seaborn -q
print("✅ Packages installed!")

In [ ]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from typing import List, Dict, Optional
from urllib.parse import urljoin
import matplotlib.pyplot as plt
import seaborn as sns

print("✅ Libraries imported!")

In [ ]:
# Configuration
CONFIG = {
    'MAX_PAGES': 5,
    'DELAY_SECONDS': 1,
    'BASE_URL': 'https://auto.amag.ch'
}

FILTERS = {
    'min_price': 400,
    'max_price': 1500,
    'fuel_types': [],  # e.g., ['Electric', 'Hybrid']
    'brands': []       # e.g., ['audi', 'bmw']
}

print(f"Config: {CONFIG}")
print(f"Filters: {FILTERS}")

In [ ]:
# AMAG Scraper Class
class AMAGScraper:
    def __init__(self, base_url, delay=1):
        self.base_url = base_url
        self.delay = delay
        
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        })
    
    def scrape_page(self, page_num):
        print(f"Scraping page {page_num}...", end=" ")
        
        url = f"{self.base_url}/de"
        params = {
            'only_abo_vehicle': '1',
            'page': page_num,
            'limit': '24',
            'sort': 'ID_DESC'
        }
        
        try:
            response = self.session.get(url, params=params, timeout=15)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.content, 'html.parser')
            cars = self.extract_cars(soup)
            
            print(f"Found {len(cars)} cars")
            time.sleep(self.delay)
            return cars
            
        except Exception as e:
            print(f"Error: {e}")
            return []
    
    def extract_cars(self, soup):
        cars = []
        
        # Find price elements to locate car containers
        price_elements = soup.find_all(text=re.compile(r'CHF\s*[\d\'.,]+'))
        
        for price_element in price_elements:
            container = price_element.parent
            
            # Go up a few levels to find main container
            for _ in range(5):
                if container and container.parent:
                    container = container.parent
                else:
                    break
            
            if container:
                car_data = self.extract_car_data(container)
                if car_data:
                    cars.append(car_data)
        
        # Remove duplicates
        unique_cars = []
        seen = set()
        for car in cars:
            car_key = (car.get('monthly_price'), car.get('title', '')[:50])
            if car_key not in seen:
                unique_cars.append(car)
                seen.add(car_key)
        
        return unique_cars[:24]  # Limit to page size
    
    def extract_car_data(self, container):
        car = {}
        all_text = container.get_text().strip()
        
        # Extract price
        price_match = re.search(r'CHF\s*([\d\'.,]+)', all_text)
        if price_match:
            try:
                price_str = price_match.group(1).replace("'", "").replace(",", "")
                car['monthly_price'] = int(price_str)
            except ValueError:
                pass
        
        # Extract fuel type
        fuel_keywords = {
            'elektrisch': 'Electric',
            'electric': 'Electric',
            'diesel': 'Diesel',
            'benzin': 'Petrol',
            'hybrid': 'Hybrid'
        }
        
        for keyword, fuel_type in fuel_keywords.items():
            if keyword.lower() in all_text.lower():
                car['fuel_type'] = fuel_type
                break
        
        # Extract transmission
        if 'automatik' in all_text.lower():
            car['transmission'] = 'Automatic'
        elif 'schaltgetriebe' in all_text.lower():
            car['transmission'] = 'Manual'
        
        # Extract title
        title_elem = container.find(['h1', 'h2', 'h3', 'h4'])
        if not title_elem:
            title_elem = container.find('a')
        if title_elem:
            car['title'] = title_elem.get_text().strip()[:100]
        
        # Extract URL
        link_elem = container.find('a', href=True)
        if link_elem:
            car['detail_url'] = urljoin(self.base_url, link_elem['href'])
        
        return car if len(car) >= 2 else None
    
    def scrape_all(self, max_pages):
        all_cars = []
        
        for page in range(1, max_pages + 1):
            cars = self.scrape_page(page)
            if not cars:
                print("No more cars found, stopping")
                break
            all_cars.extend(cars)
        
        return all_cars

print("✅ Scraper class defined!")

In [ ]:
# Run the scraper
print("🚀 Starting scrape...")
scraper = AMAGScraper(CONFIG['BASE_URL'], CONFIG['DELAY_SECONDS'])
cars_data = scraper.scrape_all(CONFIG['MAX_PAGES'])

print(f"\n✅ Scraped {len(cars_data)} total cars")

# Convert to DataFrame
if cars_data:
    df = pd.DataFrame(cars_data)
    print(f"📊 DataFrame shape: {df.shape}")
    print(f"🏷️  Columns: {list(df.columns)}")
else:
    df = pd.DataFrame()
    print("❌ No data scraped")

In [ ]:
# Apply filters
if not df.empty:
    filtered_df = df.copy()
    
    # Price filters
    if 'monthly_price' in filtered_df.columns:
        if FILTERS['min_price']:
            filtered_df = filtered_df[filtered_df['monthly_price'] >= FILTERS['min_price']]
        if FILTERS['max_price']:
            filtered_df = filtered_df[filtered_df['monthly_price'] <= FILTERS['max_price']]
    
    # Fuel type filter
    if FILTERS['fuel_types'] and 'fuel_type' in filtered_df.columns:
        filtered_df = filtered_df[filtered_df['fuel_type'].isin(FILTERS['fuel_types'])]
    
    # Brand filter
    if FILTERS['brands'] and 'title' in filtered_df.columns:
        brand_mask = filtered_df['title'].str.lower().str.contains('|'.join(FILTERS['brands']), na=False)
        filtered_df = filtered_df[brand_mask]
    
    print(f"🔍 After filtering: {len(filtered_df)} cars")
    
    # Display sample
    if not filtered_df.empty:
        print("\n📋 Sample results:")
        for i, (_, car) in enumerate(filtered_df.head(5).iterrows(), 1):
            title = car.get('title', 'Unknown')[:50]
            price = car.get('monthly_price', 'N/A')
            fuel = car.get('fuel_type', 'N/A')
            print(f"{i}. {title} - CHF {price}/month ({fuel})")
else:
    filtered_df = pd.DataFrame()

In [ ]:
# Data analysis and visualization
if not filtered_df.empty:
    print("📊 Data Analysis:")
    
    # Price statistics
    if 'monthly_price' in filtered_df.columns:
        prices = filtered_df['monthly_price'].dropna()
        print(f"💰 Price range: CHF {prices.min()} - CHF {prices.max()}")
        print(f"💰 Average: CHF {prices.mean():.0f}")
        print(f"💰 Median: CHF {prices.median():.0f}")
        
        # Price histogram
        plt.figure(figsize=(10, 6))
        plt.hist(prices, bins=20, alpha=0.7, color='skyblue', edgecolor='black')
        plt.title('Price Distribution', fontsize=16)
        plt.xlabel('Monthly Price (CHF)')
        plt.ylabel('Number of Cars')
        plt.grid(axis='y', alpha=0.3)
        plt.show()
    
    # Fuel type distribution
    if 'fuel_type' in filtered_df.columns:
        fuel_counts = filtered_df['fuel_type'].value_counts()
        print(f"\n⛽ Fuel types:")
        for fuel, count in fuel_counts.items():
            print(f"   {fuel}: {count} cars")
        
        # Fuel type pie chart
        if len(fuel_counts) > 1:
            plt.figure(figsize=(8, 8))
            plt.pie(fuel_counts.values, labels=fuel_counts.index, autopct='%1.1f%%')
            plt.title('Fuel Type Distribution')
            plt.show()
else:
    print("❌ No data for analysis")

In [ ]:
# Export data
if not filtered_df.empty:
    timestamp = pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')
    filename = f'amag_cars_{timestamp}.csv'
    
    filtered_df.to_csv(filename, index=False)
    print(f"✅ Data exported to: {filename}")
    
    # Show best deals
    if 'monthly_price' in filtered_df.columns:
        best_deals = filtered_df.nsmallest(5, 'monthly_price')
        print(f"\n💎 Top 5 Best Deals:")
        for i, (_, car) in enumerate(best_deals.iterrows(), 1):
            title = car.get('title', 'Unknown')[:40]
            price = car.get('monthly_price', 'N/A')
            fuel = car.get('fuel_type', 'N/A')
            print(f"{i}. {title} - CHF {price}/month ({fuel})")
    
    print(f"\n📥 Download the CSV file from the file browser on the left!")
else:
    print("❌ No data to export")

## Usage Notes:

### Configuration:
- Modify `MAX_PAGES` to scrape more data
- Adjust `DELAY_SECONDS` to be more respectful
- Update `FILTERS` to find specific cars

### Filters:
- `min_price` / `max_price`: Price range in CHF
- `fuel_types`: List like `['Electric', 'Hybrid']`
- `brands`: List like `['audi', 'bmw', 'tesla']`

### Export:
- CSV files are automatically timestamped
- Use file browser to download results

### Legal:
- Includes respectful delays
- Check AMAG terms of service
- Use data responsibly